In [ ]:
from IPython.core.debugger import set_trace

In [2]:
with open('/home/concys/ner-harem/test_2018-9-5_fix.iob') as f:
#with open('output/test_dataset_docid.pos') as f:
    documents = []
    paragraphs = []
    sentences = []
    tokens = []
    poses = []
    
    for line in f:
        line = line[:-1]
        #if "PPM', 'PNOUN'" in line:
        #    set_trace()
        
        if line != 'S' and line != 'P' and not line.startswith('D'):
            tokens.append(line.split(', ')[0][1:-1])
            poses.append(line.split(', ')[1][1:-1])
        elif line == 'S':
            sentences.append(list(zip(tokens, poses)))
            tokens = []
            poses = []
        elif line == 'P':
            paragraphs.append(sentences)
            sentences = []
        elif line.startswith('D'):
            docid = line.split()[1]
            documents.append((paragraphs, docid))
            paragraphs = []
            

In [3]:
#documents = documents[0:1]

In [4]:
def addCharInformation(sentences):
    new_sentences = []
    for s in sentences:
        new_sentence = []
        for data in s:
            token = data[0]
            chars = [c for c in token]
            new_sentence.append([data[0], data[1], chars])
        new_sentences.append(new_sentence)
    return new_sentences

In [5]:
from keras.preprocessing.sequence import pad_sequences
import keras

def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def createMatrices(sentences, word2Idx, case2Idx,char2Idx, pos2Idx):
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
        posIndices = []
        
        for word, pos, char in sentence: 
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                # makes an exception for non-breaking space
                # add this character to char2Idx later
                x = x.replace('\xa0', ' ')
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
            posIndices.append(pos2Idx[pos])
           
        dataset.append([wordIndices, caseIndices, charIndices, posIndices]) 
        
    return dataset

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

Using TensorFlow backend.


In [6]:
import pickle

word2Idx = pickle.load(open('word2Idx.pkl', 'rb'))
case2Idx = pickle.load(open('case2Idx.pkl', 'rb'))
char2Idx = pickle.load(open('char2Idx.pkl', 'rb'))
pos2Idx = pickle.load(open('pos2Idx.pkl', 'rb'))
idx2Label = pickle.load(open('idx2L.pkl', 'rb'))


In [7]:
#matrices = padding(createMatrices(char_sentences, word2Idx, case2Idx, char2Idx, pos2Idx))


In [8]:
#model = keras.models.load_model('/home/concys/ner-harem/model_with_pos_v2.bin')
model = keras.models.load_model('/home/concys/ner-harem/model_without_pos.bin')

In [9]:
import numpy as np

def predict_labels(matrices):
    labels = []
    for data in matrices:
        #print(data)
        tokens, casing, char, pos = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pos = np.asarray([pos])

        #print('tokens', tokens)
        #print('casing', casing)
        #print('char', char)
        #print('pos', pos)



        #pred = model.predict([tokens, casing, pos, char], verbose=False)[0]   
        pred = model.predict([tokens, casing, char], verbose=False)[0]
        pred = pred.argmax(axis=-1) #Predict the classes
        #print('Labels:')
        #print(pred)
        labels.append([idx2Label[p] for p in pred])
        
    return labels


In [10]:
def annotate_text(labels, id_counter, text, out_tree):
    has_found_em = False
    is_outside_em = True
    for label, token in zip(labels, text):
        #label = label[0]
        if label.startswith('B_'):
            category = label.split('_')[1]
            out_em = ET.SubElement(out_tree, 'EM', attrib={'ID': str(id_counter), 'CATEG': category})
            out_em.tail = ''
            id_counter += 1
            out_em.text = token
            has_found_em = True
            is_outside_em = False
        elif label.startswith('I_'):
            if not is_outside_em:
                out_em.text += ' ' + token
            else:
                category = label.split('_')[1]
                out_em = ET.SubElement(out_tree, 'EM', attrib={'ID': str(id_counter), 'CATEG': category})
                out_em.tail = ''
                id_counter += 1
                out_em.text = ' ' + token
            has_found_em = True
        else:
            if not is_outside_em:
                is_outside_em = True
                out_em.tail += ' ' + token
            else:
                if not has_found_em:
                    if out_tree.text:
                        out_tree.text += ' ' + token
                    else:
                        out_tree.text = token
                else:
                    out_em.tail += ' ' + token
                
    return id_counter

In [11]:
#for s, l in zip(sentences, labels):
#    for token, l_token in zip(s, l):
#        print(token[0], l_token)

In [12]:
import xml.etree.ElementTree as ET

id_counter = 0
out_tree = ET.Element('colHAREM')
out_tree.set('versao', 'ColeccaoSegundoHAREM-4.0')
for document in documents:
    out_doc = ET.SubElement(out_tree, 'DOC')
    # add DOCID later
    out_doc.set('DOCID', document[1])
    for p in document[0]:
        out_p = ET.SubElement(out_doc, 'P')
        
        sentences = p
        #if any(('PPM', 'PNOUN') in s for s in sentences):
            
        #    set_trace()
        matrices = padding(createMatrices(addCharInformation(sentences), word2Idx, case2Idx, char2Idx, pos2Idx))

        labels = predict_labels(matrices)
        labels = [i for l in labels for i in l]

        
        #print(labels)
        
        text = [i[0] for l in sentences for i in l]
        #print(text)
        id_counter = annotate_text(labels, id_counter, text, out_p)
        

In [13]:
ot = ET.ElementTree(out_tree)

ot.write('harem_without_pos_fix.xml', encoding='UTF-8')

In [ ]:
matrices = padding(createMatrices(addCharInformation([['O', 'Pedro', 'mora', 'em', 'Porto', 'Alegre', 'e', 'estuda', 'na', 'UFRGS', '.']]), word2Idx, case2Idx, char2Idx, pos2Idx))
labels = predict_labels(matrices)

print(labels)
